In [25]:
import heapq 
from collections import Counter

# Final Project Huffman Compression Code
### Regular huffman encoding/decoding


In [26]:
# Takes in file and counts frequencies of each character in the file
# Planning on sending in War and Peace txt, which is a large sample of the English language
def count_frequencies(pathname):
    freq = {}
    with open(pathname, "r") as f:
        for char in f.read():
            freq[char] = freq.get(char, 0) + 1
    return freq

# Class for a binary tree node. Holds a character, frequency of that character, and two children
class BTNode:
    def __init__(self, char=None, freq=0):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    # "Magic" function that allows you to use it in a priority queue
    # We are using a min queue so want to pull out lowest frequency first
    def __lt__(self, other):
        return self.freq < other.freq

# Builds the regular huffman tree from a dictionary (freqs) 
# Returns the tree and freqs for easy access
def build_huffman_tree(freqs):
    # Make a priority queue which is heapified by minimum frequency 
    priority_queue = [BTNode(ch, f) for ch, f in freqs.items()]
    heapq.heapify(priority_queue)

    # loop while it is length greater than 1, pulling out two elements and then merging them
    while(len(priority_queue) > 1):
        t1 = heapq.heappop(priority_queue)
        t2 = heapq.heappop(priority_queue)
        tree = BTNode(None, t1.freq + t2.freq) # make new tree with t1 on left, t2 on right
        tree.left = t1
        tree.right = t2
        heapq.heappush(priority_queue, tree) # add the tree back onto the priority queue and loop
    huff_tree = priority_queue[0] # return the the one remaining node, the priority tree
    return huff_tree

# recursively builds the codebook, which maps characters to their codes 
def build_codebook(node, prefix = "", codebook = None):
    if codebook is None: 
        codebook = {}
    if node.char is not None: # Base case: if we are at a leaf, add the 
        codebook[node.char] = prefix
    else:
        build_codebook(node.left, prefix + "0", codebook)
        build_codebook(node.right, prefix + "1", codebook)
    return codebook

# encodes a message given a particular codebook
def encode_message(message, codes):
    encoded_message = "".join(codes[ch] for ch in message) # uses generator expression to create encoded message
    return encoded_message

# decodes a message given a particular huffman tree (required for decoding)
def decode_message(encoded_message, huff_tree):
    result = [] # holds decoded message
    node = huff_tree
    for bit in encoded_message:
        node = node.left if bit == "0" else node.right # go left if 0, go right if 1
        if node.char is not None:  # once you get to leaf
            result.append(node.char)  # add the correct character to the decoded message
            node = huff_tree # start back at the top for next iteration
    decoded_message = "".join(result)
    return decoded_message

def compute_exp_bits(huffman_tree, freqs):
    import math
import heapq

# def compute_exp_bits(huffman_tree, freqs):
#     """Compute expected code length and entropy given a Huffman tree and freqs"""
#     # First build code lengths
#     codebook = build_codebook(huffman_tree)
#     lengths = {ch: len(code) for ch, code in codebook.items()}

#     total = sum(freqs.values())
#     probs = {ch: freq / total for ch, freq in freqs.items()}

#     # Expected bits per symbol
#     exp_bits = sum(probs[ch] * lengths[ch] for ch in probs)

#     # Entropy of the source
#     entropy = -sum(p * math.log2(p) for p in probs.values())

#     return exp_bits, entropy, lengths
# Test regular Huffman tree

freqs = count_frequencies("WarAndPeace.txt")

test_text = "hello"
basic_tree = build_huffman_tree(freqs)
basic_codes = build_codebook(basic_tree)
encoded = encode_message(test_text, basic_codes)
print("Basic Codes:", basic_codes)
print("Message:", test_text)
print("Encoded:", encoded)
print("Decoded:", decode_message(encoded, basic_tree))

Basic Codes: {'e': '000', 's': '0010', 'h': '0011', 'i': '0100', 'p': '010100', ',': '010101', ':': '01011000000', '6': '010110000010000', '3': '010110000010001', '/': '0101100000100100', '=': '0101100000100101000', ']': '01011000001001010010', '#': '01011000001001010011', '[': '01011000001001010100', '%': '01011000001001010101', '$': '01011000001001010110', '@': '01011000001001010111', '4': '01011000001001011', '9': '0101100000100110', 'Q': '0101100000100111', 'U': '0101100000101', '*': '0101100000110', 'J': '0101100000111', 'j': '0101100001', 'z': '0101100010', 'q': '0101100011', ';': '01011001000', 'K': '01011001001', 'Y': '01011001010', 'G': '01011001011', 'R': '0101100110', '2': '01011001110000', '0': '01011001110001', 'X': '0101100111001', ')': '010110011101', '(': '010110011110', 'L': '010110011111', 'W': '0101101000', 'S': '0101101001', '-': '010110101', 'P': '010110110', 'A': '010110111', 'y': '010111', 'n': '0110', 'o': '0111', 'l': '10000', 'g': '100010', 'v': '1000110', '?'

### Add new codes for bigrams, which allow for characters like "e " which are very common in the English language. 

In [27]:
# Computes the expected bits per character
# bits per symbol = sum of prob(s) * length(code(s))
def expected_bits(freqs, codebook):
    total = sum(freqs.values()) # total number of characters
    bits_per_symbol = sum((freqs[s]/total) * len(codebook[s]) for s in freqs) # bits per symbol = sum of prob(s) * length(code(s))
    avg_chars = sum((freqs[s]/total) * (2 if len(s)==2 else 1) for s in freqs) # average originial characters per symbol is key becaus you have more total symbols with bigrams
    return bits_per_symbol / avg_chars # division gives bits per original character


# Identifies the top K bigrams from the text in terms of freqeuncy
def top_bigrams(text, K):
    counts = Counter(text[i:i+2] for i in range(len(text)-1)) # uses Counter package to count occurences of bigrams in text
    return set(bg for bg,_ in counts.most_common(K)) # returns the K most common bigrams as a set

# walks through the text and returns a list of symbols ready for Huffman counting to make into a tree

def replace_bigrams(text, bigram_set):
    out = []
    i = 0
    while i < len(text):
        if i+1 < len(text) and text[i:i+2] in bigram_set: # look at next 2 elements, if in bigram set, add that combination to list
            out.append(text[i:i+2])
            i += 2
        else: # else, just add the character normally
            out.append(text[i])
            i += 1
    return out

text = "this is a simple example to test bigram compression and see the effect on bits"
K = 1000000000000000000000000
bigrams = top_bigrams(text, K)
mixed = replace_bigrams(text, bigrams)
freqs = Counter(mixed)
tree = build_huffman_tree(freqs)
codebook = build_codebook(tree)
bits_per_char = expected_bits(freqs, codebook)

print("Top bigrams:", bigrams)
print("Huffman codebook:", codebook)
print("Expected bits per original character:", bits_per_char)


Top bigrams: {'se', 'ee', 'ra', 'st', 'ff', 'mp', 'e ', 'pl', 'to', 'om', ' c', 'pr', 'ts', 't ', 'on', 'd ', ' o', 'th', 'ef', 'hi', 'io', ' e', 'an', 'le', 'ct', 'it', 'a ', 'es', 'im', 'he', 'ig', 'ss', ' t', ' s', 'fe', 'nd', 's ', ' i', 'xa', 'bi', 'si', ' a', 'm ', 'am', 'gr', 'o ', ' b', 'n ', 'ex', 'co', 'te', 'ec', 'is', 're'}
Huffman codebook: {'co': '00000', ' e': '00001', 'e ': '0001', 'ct': '00100', ' i': '00101', 'ef': '00110', 'se': '00111', 'an': '01000', 'io': '01001', 'is': '01010', 'ss': '01011', 'xa': '01100', 're': '01101', 'le': '0111', ' o': '10000', ' b': '10001', 'n ': '1001', 'ra': '10100', 'ts': '10101', 'fe': '10110', 'st': '10111', 'mp': '1100', 'th': '11010', 'ig': '110110', 'bi': '110111', 'd ': '111000', 'o ': '111001', 's ': '111010', 'm ': '111011', 'te': '111100', 'si': '111101', ' t': '111110', 'a ': '111111'}
Expected bits per original character: 2.5128205128205128
